In [1]:
from netgen.occ import *
from ngsolve.meshes import MakeStructured3DMesh
from ngsolve import *

l = 200
b = 40
t = 10
h = 1

mesh = MakeStructured3DMesh(hexes=False,nx=40*h,ny=16*h,nz=2*h, mapping=lambda x,y,z : (-l+l*2*x,(-b+2*b*y)*(3-2*(-1+2*x)**2),t*z - 7*t*sin(2*(-1+2*x))), secondorder = True)

from ngsolve.webgui import Draw

Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [2]:
# Projections
n = specialcf.normal(mesh.dim)

def Anti(vec):
    return CF( (0, -vec[2], vec[1],
                vec[2], 0, -vec[0],
                -vec[1], vec[0], 0), dims = (3,3) ) 

def Dyad(v1, v2):
    return CF( (v1[0]*v2[0], v1[0]*v2[1], v1[0]*v2[2],
                v1[1]*v2[0], v1[1]*v2[1], v1[1]*v2[2],
                v1[2]*v2[0], v1[2]*v2[1], v1[2]*v2[2]), dims = (3,3) ) 

Antin = Anti(n)
Q = Dyad(n, n)
P = Id(3) - Q

# Weingarten tensor
W = -Grad(n)*P

In [3]:
# Volume material: silicone rubber 
mue = 0.34 
muc = 1
lame = 5.328 
Lc = 10**-2

def Ce(mat):
    return 2*mue*mat + lame*Trace(mat)*Id(3)


# Shell material: graphite 1.6 mm
h = 1.6
mues = 2122.64 
lames = 289.451
mucs = 10000


# Plane stress
lames = 2*lames*mues / (lames + 2*mues)


def De(mat):
    return 2*mues*mat + lames*Trace(mat)*P
    

muelca1 = 10867.9
muelca2 = 122264
muelca3 = 0


def Vol(mat):
    return (1/3) * Trace(mat) * Id(3) 


def Dev(mat):
    return mat - Vol(mat)


def L(mat):
    return  muelca1 * Dev(Sym(mat)) + muelca2 * Skew(mat) + muelca3 * Vol(mat)


uD = CF ( (0,0,0), dims = (3,1) )
rD = CF ( (0,0,0), dims = (3,1) )

q = 1e-6 * CF ( (0,0,-1), dims = (3,1) )
m = CF ( (0,0,0), dims = (3,1) )

In [4]:
order = 3
fesHu = VectorH1(mesh, order=order, dirichlet="back|front")
fesHr = VectorH1(mesh, order=order) # , dirichlet="back|front"

fes = fesHu * fesHr

(u, r), (du, dr) = fes.TnT()

Dr = Grad(r)
Du = Grad(u)

Dtr = Dr.Trace() * P
Dcovr = P * Dtr

SymDu = Sym(Du)
SkewDu = Skew(Du)

Dtu = Grad(u).Trace() * P
Dcovu = P * Dtu

SymDcovu = Sym(Dcovu)  
SkewDcovu = Skew(Dcovu)

R = Anti(r)
CurlR = Trace(Dr)*Id(3) - Dr.trans

Rt = R * P
Rcov = P * Rt

CurltR = InnerProduct(Dr.Trace(), P)*Id(3) - P * Dr.Trace().trans

In [5]:
a = BilinearForm(fes, symmetric=True, symmetric_storage=True, condense=True)

# Volume 
a += (Variation(0.5*(
    InnerProduct(SymDu,Ce(SymDu)) 
    + 2*muc * InnerProduct(SkewDu - R, SkewDu - R)
    + mue*Lc**2 * InnerProduct(CurlR, CurlR)
)*dx))

# Shell
a += Variation(0.5*(
    h * InnerProduct(SymDcovu,De(SymDcovu)) 
    + h * 2*mucs * InnerProduct(SkewDcovu - Rcov, SkewDcovu - Rcov)
    + h * (mues + mucs) * InnerProduct(Q*(Dtu - Rt), Q*(Dtu - Rt))
    + (h**3 / 12) * InnerProduct(Sym(Antin * Dcovr + P*R*W), De(Sym(Antin * Dcovr + P*R*W)))
    + (h**3 / 12) * 2*mucs * InnerProduct(Skew(Antin * Dcovr + P*R*W), Skew(Antin * Dcovr + P*R*W))
    + (h**3 / 12) * (mues + mucs) * InnerProduct(Q*R*W, Q*R*W) 
    + h * InnerProduct(CurltR, L(CurltR))
)*ds("bottom"))

# Micro shell
# a += Variation(0.5*(
#     h * InnerProduct(SymDcovu,De(SymDcovu)) 
#     + h * 2*mucs * InnerProduct(SkewDcovu - Rcov, SkewDcovu - Rcov)
#     + h * (mues + mucs) * InnerProduct(Q*(Dtu - Rt), Q*(Dtu - Rt))
#     + h * InnerProduct(CurltR, L(CurltR))
# )*ds("bottom"))
 
f = LinearForm(fes)
f += (InnerProduct(du,q) + InnerProduct(dr,m))*dx

sol = GridFunction(fes)

In [6]:
sol.vec[:] = 0

sol.components[0].Set( uD )
sol.components[1].Set( rD )

v = sol.vec.CreateVector()
w = sol.vec.CreateVector()

In [7]:
with TaskManager():
    f.Assemble()
    a.AssembleLinearization(sol.vec)
    inv = a.mat.Inverse(fes.FreeDofs(a.condense), inverse="pardiso")

    a.Apply(sol.vec, v)
    
    v.data -= f.vec
    if a.condense:
        v.data += a.harmonic_extension_trans * v
    w.data = inv * v
    if a.condense:
        w.data += a.harmonic_extension * w
        w.data += a.inner_solve * v
    sol.vec.data -= w

In [8]:
gfu, gfr = sol.components

Draw(gfu, mesh, "u", deformation=True)
# Draw(Norm(gfr), mesh, "r")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [9]:
# vtk = VTKOutput(ma=mesh,
#                 coefs=[gfu],
#                 names = ["displacement"],
#                 filename="disp-cosserat-microshell",
#                 subdivision=3)
# # Exporting the results:
# vtk.Do()